In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import os
import glob

In [2]:
d_gcs=pd.read_csv("gcs_scores_updated.csv")
d_gcs=d_gcs.drop(columns=['nursingchartcelltypecat','nursingchartcelltypevallabel'])
d_gcs=d_gcs.drop_duplicates()
d_gcs=d_gcs.rename(columns={'nursingchartentryoffset':'offset'})

labs=pd.read_csv("labs_morevars.csv")
labs=labs.drop_duplicates()
labs=labs.rename(columns={'labresultoffset':'offset'})

cardiovas=pd.read_csv("drugate_norm_updated.csv")
cardiovas.drop_duplicates()
cardiovas=cardiovas.drop(columns=['nursingchartvalue','SOFA_cardio'])

In [3]:
#First merge
labs_cardio=pd.merge(labs,cardiovas,how="outer",on=['patientunitstayid','offset']).drop_duplicates()
labs_cardio.to_csv("labs_cardio_interim.csv",index=False)

del labs
del cardiovas

In [4]:
training_build=pd.merge(labs_cardio,d_gcs,how="outer",on=['patientunitstayid','offset']).drop_duplicates()
del d_gcs

In [5]:
#Initial length due to replication
len(training_build)

16406206

In [6]:
#To correct the replication of offsets for same patients
training_build=training_build.groupby(['patientunitstayid','offset'],as_index=False).max().drop_duplicates()

In [7]:
len(training_build)

9536931

In [8]:
training_build=training_build.groupby(['patientunitstayid'],as_index=False).apply(pd.DataFrame.sort_values,'offset').reset_index()
training_build=training_build.drop(columns=['level_0','level_1'])
training_build=training_build.drop(columns=['Norepinephrine','Epinephrine','Dopamine','Dobutamine'])
training_build_filtered=training_build.dropna(subset=['paO2_FiO2','platelets_x_1000','total_bilirubin','urinary_creatinine','creatinine','HCO3','pH','paCO2','direct_bilirubin','excess','ast','bun','calcium','GCS_Score'],how='all')
training_build_filtered['label']=np.nan

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [9]:
sepsis_labels=pd.read_csv("24_hour_sepsis.csv")
sepsis_labels['tsepsis']=sepsis_labels[['tsus','tsofa']].min(axis=1)

In [10]:
final_build=pd.merge(training_build_filtered,sepsis_labels,how='left',left_on=['patientunitstayid','offset'],right_on=['patientunitstayid','tsepsis'])
final_build.label=final_build.flag
final_build=final_build.drop(columns=['tsofa','tsus','tsepsis','flag'])
#After the initial sepsis=1 flag, all the labels for that patient is given label=1, all before that is 0
final_build['label']=final_build.groupby(['patientunitstayid'])['label'].ffill()
final_build['label']=final_build['label'].fillna(0)

In [11]:
final_build.to_csv("training_table.csv",index=False)

In [12]:
final_build_nonneg=final_build.loc[~(final_build['offset']<0)]
final_build_nonneg.to_csv("training_table_nonneg.csv",index=False)

In [15]:
len(final_build_nonneg['patientunitstayid'].unique())

191623

In [13]:
pids=withoutvital.patientunitstayid.unique()

In [3]:
#we need to add vitals as well for these patients
withoutvital=pd.read_csv("training_table_nonneg.csv")
pids=withoutvital.patientunitstayid.unique()
vitals=pd.read_csv("vitalPeriodic.csv",chunksize=10000,usecols=['patientunitstayid','observationoffset','heartrate','respiration','temperature'])
chunks=[]
def processchunk(chunk):
    tomerge=chunk.loc[chunk['patientunitstayid'].isin(pids)]
    chunks.append(tomerge)
    

In [4]:
for chunk in vitals:
    processchunk(chunk)

In [5]:
vitals_tomerge=pd.concat(chunks,sort=False)
vitals_tomerge=vitals_tomerge.rename(columns={'observationoffset':'offset'})

In [6]:
len(vitals_tomerge['patientunitstayid'].unique())
del withoutvital
del chunks
del chunk
vitals_tomerge.to_csv("vitals_tomerge.csv",index=False)
del vitals_tomerge


In [2]:
# Read in the csv files.
train1 = dd.read_csv('training_table_nonneg.csv')
vitals = dd.read_csv('vitals_tomerge.csv')

# Merge the csv files.
train2 = dd.merge(train1, vitals, how='outer', on=['patientunitstayid','offset'], npartitions=25)



In [3]:
# Write the output.
train2.to_csv('training_with_vitals-*.csv', index=False)

['training_with_vitals-00.csv',
 'training_with_vitals-01.csv',
 'training_with_vitals-02.csv',
 'training_with_vitals-03.csv',
 'training_with_vitals-04.csv',
 'training_with_vitals-05.csv',
 'training_with_vitals-06.csv',
 'training_with_vitals-07.csv',
 'training_with_vitals-08.csv',
 'training_with_vitals-09.csv',
 'training_with_vitals-10.csv',
 'training_with_vitals-11.csv',
 'training_with_vitals-12.csv',
 'training_with_vitals-13.csv',
 'training_with_vitals-14.csv',
 'training_with_vitals-15.csv',
 'training_with_vitals-16.csv',
 'training_with_vitals-17.csv',
 'training_with_vitals-18.csv',
 'training_with_vitals-19.csv',
 'training_with_vitals-20.csv',
 'training_with_vitals-21.csv',
 'training_with_vitals-22.csv',
 'training_with_vitals-23.csv',
 'training_with_vitals-24.csv']

In [3]:
extension = 'csv'
all_filenames = [i for i in glob.glob('training_with_vitals*.{}'.format(extension))]

In [5]:
with open('combined_train_vitals.csv', 'w') as outfile:
    for in_filename in all_filenames:
        with open(in_filename, 'r') as infile:
            # if your csv files have headers then you might want to burn a line here with `next(infile)
            next(infile)
            for line in infile:
                outfile.write(line + '\n')

In [2]:
new_train=pd.read_csv("combined_train_vitals.csv",chunksize=100000,header=None)
df_new=[]
for chunk in new_train:
    chunk.iloc[:,2:]=chunk.iloc[:,2:].astype('float16')
    chunk.iloc[:,:2]=chunk.iloc[:,:2].astype('int32')
    df_new.append(chunk)
training_data=pd.concat(df_new,sort=False)
del df_new
del chunk

In [8]:
training_data.head()

,patientunitstayid,labresultoffset,paO2_FiO2,platelets_x_1000,total_bilirubin,urinary_creatinine,creatinine,HCO3,pH,paCO2,...,hgb,ptt,wbc,fibrinogen,troponin,GCS_Score,label,heartrate,respiration,temperature
0,141233,692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,0.0,NaN,NaN,NaN
1,141233,6744,NaN,NaN,NaN,NaN,0.910156,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
2,141233,7892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,15.0,0.0,NaN,NaN,NaN
3,141233,11477,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,15.0,0.0,NaN,NaN,NaN
4,141244,420,NaN,84.0,NaN,NaN,0.649902,NaN,NaN,NaN,...,9.898438,NaN,4.101562,NaN,NaN,NaN,0.0,NaN,110.0,13.0


In [29]:
cols=['patientunitstayid','offset','paO2_FiO2','platelets_x_1000',
      'total_bilirubin','urinary_creatinine','creatinine','HCO3','pH','paCO2',
      'direct_bilirubin','excess','ast','bun','calcium','glucose','lactate',
      'magnesium','phosphate','potassium','hct','hgb','ptt','wbc','fibrinogen','troponin','GCS_Score','label',
      'heartrate','respiration','temperature']

training_data.columns=cols

In [ ]:
training_data.describe()

In [27]:
#change the datatype of the columns which have max less than 255 and min greater -256 to float8
training_data[['GCS_Score','label']]=training_data[['GCS_Score','label']].astype('int8')

ValueError: Cannot convert non-finite values (NA or inf) to integer

In [28]:
training_data=training_data.groupby(['patientunitstayid','offset'],as_index=False).max().drop_duplicates()

KeyError: 'offset'

patientunitstayid       int32
labresultoffset         int32
paO2_FiO2             float16
platelets_x_1000      float16
total_bilirubin       float16
urinary_creatinine    float16
creatinine            float16
HCO3                  float16
pH                    float16
paCO2                 float16
direct_bilirubin      float16
excess                float16
ast                   float16
bun                   float16
calcium               float16
glucose               float16
lactate               float16
magnesium             float16
phosphate             float16
potassium             float16
hct                   float16
hgb                   float16
ptt                   float16
wbc                   float16
fibrinogen            float16
troponin              float16
GCS_Score             float16
label                 float16
heartrate             float16
respiration           float16
temperature           float16
dtype: object